In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.translate.bleu_score import sentence_bleu


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
full_data = pd.read_csv("../input/inspoquotesgenerated/InspoQuotes.csv")
full_data.head()

In [ ]:
full_data.shape

In [ ]:
train_data = full_data.sample(n=39)

In [ ]:
def cleanText(text):
    text.split()
    return text

In [ ]:
lst = ['No safety, No ejaculation', 'Some people look for a beautiful place. Others make a place beautiful.', "Happiness often sneaks in through a door you didn't know you left open.", 'We must be willing to let go of the life we planned so as to have the life that is waiting for us.', 
            'Those who lie about laziness, Objectify light.', 'Be productive and be yourself', 'You can be an unhealthy animal. Weird.','Life is 10 percent what happens to me and 90 percent of how I react to it.','The fact youre hairy does not make you inexplicable']
  
# Calling DataFrame constructor on list
test_data = pd.DataFrame(lst, columns = ['Quote']) 

In [ ]:
print(train_data.shape)
train_data.head()

In [ ]:
print(test_data.shape)
test_data.head()

In [ ]:
BLEUscore_Gendata = test_data
bg = BLEUscore_Gendata["Quote"]
bg.shape

BLEUscore_data = train_data.loc[train_data['Generated'] == 0, ['Quote']]
b = BLEUscore_data["Quote"]
b = b.str.split()
b.shape

lstBLEU = []
i = 0
for element in bg:
    p = element.split()
    i = i + 1
    lstBLEU.append(sentence_bleu(b, p))
lstBLEU

In [ ]:
def extract_features(full_data,train_data,test_data):
               
    # TF-IDF BASED FEATURE REPRESENTATION
    tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95)
    tfidf_vectorizer.fit_transform(train_data[field].values)
        
    train_feature_set=tfidf_vectorizer.transform(train_data[field].values)
    test_feature_set=tfidf_vectorizer.transform(test_data[field].values)
        
    return train_feature_set,test_feature_set,tfidf_vectorizer

In [ ]:
Y_train=train_data['Generated'].values

Y_test=pd.get_dummies(train_data['Generated'])

field = "Quote"

X_train,X_test,feature_transformer=extract_features(full_data,train_data,test_data)

In [ ]:

# INIT LOGISTIC REGRESSION CLASSIFIER
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=1, C=5, penalty='l2',max_iter=1000, dual = True)
model=scikit_log_reg.fit(X_train,Y_train)

print(model.score(X_train,Y_train))

y_predict = model.predict(X_test)

print(y_predict)

In [ ]:
print(lstBLEU)

In [ ]:
index_list = test_data.Quote.tolist()
index_list

1 0 0 0 1 1 1 0 1

In [ ]:
print(test_data)

In [ ]:
y_predict
BLEUscore_Gendata['Label'] = y_predict

In [ ]:
BLEUscore_Gendata['BLEU'] = lstBLEU

In [ ]:
BLEUscore_Gendata['Label'] = lstBLEU

In [ ]:
BLEUscore_Gendata

In [ ]:
output = pd.DataFrame({'Quote': test_data.Quote, 'Generated': y_predict})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")